**How to access an Excel file and insert it into to Database.bold text**

In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import datetime
import pandas as pd
import openpyxl
from apple_fiscal_year.fiscal_functions import AppleFiscalYear
import mysql.connector as mysql
from sqlalchemy import create_engine
from tqdm import tqdm

/var/folders/kk/m7gtvp5s53d8zr02s1ls5gfm0000gn/T/ipykernel_2503/1072392418.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
fiscal_year = AppleFiscalYear()

#Acess the excel file and make it into a dataframe.

dataframe = openpyxl.load_workbook("")

dataframe1 = dataframe.active

print(dataframe1)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<Worksheet "Query for Matt - SAP Daily Feed">


In [ ]:
list_header = ['store_num', 'date', 'billing_units', 'billing_amt_usd', 'in_count_units', 'out_count_units', 'net_transaction_amt', 'retail_transaction_units', 'total_transaction_units', 'units_per_transaction', 'created_at', 'created_by', 'fiscal_day', 'fiscal_week', 'fiscal_period', 'fiscal_quarter', 'fiscal_year', 'id']

dict = {}

def my_fun(my_list, cntr):

    #print(my_list)

    my_fiscal_date = my_list[0].date()

    my_fiscal_date = str(my_fiscal_date)

    year = fiscal_year.fiscal_year(my_fiscal_date)

    week = fiscal_year.fiscal_week(my_fiscal_date)

    period = fiscal_year.fiscal_period(my_fiscal_date)

    quarter = fiscal_year.fiscal_quarter(my_fiscal_date)

    day = fiscal_year.fiscal_day(my_fiscal_date)

    now_time = datetime.datetime.now()

    one_dict = { 'date': my_fiscal_date , 'store_num': my_list[1] ,'billing_units': my_list[3], 'billing_amt_usd':my_list[4], 'in_count_units': my_list[5], 'out_count_units':my_list[6], 'net_transaction_amt': 'NULL', 'retail_transaction_units': my_list[7], 'total_transaction_units': 'NULL', 'units_per_transaction': 'NULL', 'created_at': now_time , 'created_by': 'CronJob' , 'fiscal_day': int(day['number']), 'fiscal_week': week['number'], 'fiscal_period':period['number'], 'fiscal_quarter': quarter['number'], 'fiscal_year': year['number'], 'id': cntr + 1}

    #print(one_dict)

    dict[cntr] = one_dict



cntr = 0

for row in tqdm(range(5, dataframe1.max_row)):

    my_list_of_values = []

    for col in dataframe1.iter_cols(2, dataframe1.max_column):
        my_list_of_values.append(col[row].value)

    my_fun(my_list_of_values, cntr)
    cntr = cntr + 1



print(len(dict))

100%|███████████████████████████████████████| 3906/3906 [03:50<00:00, 16.94it/s]

3906


In [ ]:
print(dict[1]['created_at'])

2024-02-28 17:32:58.264026


In [2]:
dict[1]

dict[1]

In [1]:

connection = mysql.connect(user='root',
                           password='softzino',
                           host='127.0.0.1',
                           database='softzino')

print(dict)


NameError: name 'mysql' is not defined

In [ ]:
connection.is_connected()

print(len(list_header))

18


In [3]:
for i in tqdm(range(1 ,len(dict))):

    query = "("

    for j in range(0, len(list_header)):

        if dict[i]['billing_units'] is None:
            dict[i]['billing_units'] = 0


        if dict[i]['billing_amt_usd'] is None:
            dict[i]['billing_amt_usd'] = 0

        if dict[i]['in_count_units'] is None:
            dict[i]['in_count_units'] = 0

        if dict[i]['out_count_units'] is None:
            dict[i]['out_count_units'] = 0

        if dict[i]['retail_transaction_units'] is None:
            dict[i]['retail_transaction_units'] = 0


        if type(dict[i][list_header[j]]) == type("check") and dict[i][list_header[j]] != 'NULL':
            dict[i][list_header[j]] = f"'{dict[i][list_header[j]]}'"

        if type(dict[i][list_header[j]]) == type(datetime.date.today()):
            dict[i][list_header[j]] = f"'{dict[i][list_header[j]]}'"

        if j == len(list_header)-1:
            query += f'{dict[i][list_header[j]]}'
            continue

        query += f'{dict[i][list_header[j]]},'

    query += ")"

    main_query = f"INSERT INTO ma_sap_key_figures VALUES {query}"

    print(main_query)

    cursor = connection.cursor()
    cursor.execute(main_query)

connection.commit()

NameError: name 'tqdm' is not defined

INSERT INTO ma_sap_key_figures VALUES ('R211','2023-06-21',0,296.17,0,0,NULL,0,NULL,NULL,'2024-02-28','CronJob',270,39,9,3,2023,3)